# Perceptron Algorithm for Classification of Iris Dataset

In [67]:
%pip install xlrd
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [68]:
# load the iris dataset
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

Preprocess the data

In [69]:
# Preprocess the data
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=None)

Define the perceptron algorithm

### Adapt to shapes:

In [71]:
import numpy as np

# Define the perceptron algorithm
class MultiClassPerceptron:
    def __init__(self, input_dim, output_dim, lr=0.01, epochs=1000):
        self.W = np.random.randn(input_dim, output_dim)
        self.b = np.zeros((1, output_dim))
        self.lr = lr
        self.epochs = epochs

    def forward(self, X):
        weighted_sum = np.dot(X, self.W) + self.b
        probabilities = np.exp(weighted_sum) / np.sum(np.exp(weighted_sum), axis=1, keepdims=True)
        return probabilities

    def backward(self, X, y):
        m = X.shape[0]

        probabilities = self.forward(X)
        # Convert y to one-hot encoded form
        y_one_hot = np.eye(self.W.shape[1])[y]

        dW = (1 / m) * np.dot(X.T, (probabilities - y_one_hot))
        db = (1 / m) * np.sum(probabilities - y_one_hot, axis=0)

        self.W -= self.lr * dW
        self.b -= self.lr * db

    def fit(self, X, y):
        for epoch in range(self.epochs):
            self.forward(X)
            self.backward(X, y)

    def predict(self, X):
        probabilities = self.forward(X)
        predictions = np.argmax(probabilities, axis=1)
        return predictions


Train the model

In [72]:
# Train the model
p = MultiClassPerceptron(input_dim=X_train.shape[1], output_dim=3, lr=0.0005, epochs=1000)
p.fit(X_train, y_train)
predictions_train = p.predict(X_train)
predictions = p.predict(X_test)

Evaluate the model

In [73]:
# evaluate train accuracy
print("Perceptron classification train accuracy", accuracy_score(y_train, predictions_train))
print("Perceptron classification accuracy", accuracy_score(y_test, predictions))

Perceptron classification train accuracy 0.21666666666666667
Perceptron classification accuracy 0.3


Non-linear feature transformation on the concrete compressive strength dataset

In [75]:
import numpy as np
from itertools import combinations_with_replacement

# Implement the polynomial_features() function
def polynomial_features(X, degree):
    n_samples, n_features = X.shape
    polynomial_X = np.ones((n_samples, 1))

    for d in range(1, degree + 1):
        combinations = combinations_with_replacement(range(n_features), d)
        for comb in combinations:
            poly_features = np.prod(X[:, comb], axis=1, keepdims=True)
            polynomial_X = np.hstack((polynomial_X, poly_features))

    return polynomial_X


In [76]:
# Non-linear feature transformation
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# load the concrete compressive strength dataset
df = pd.read_excel('Concrete_Data.xls')

# Selecting the Concrete compressive strength as targets, rest is features
X = df.drop('Concrete compressive strength(MPa, megapascals) ', axis=1)
y = df['Concrete compressive strength(MPa, megapascals) ']


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=None)

# transform the features into second degree polynomial features
X_train_poly_custom = polynomial_features(X_train.values, degree=2)
X_test_poly_custom = polynomial_features(X_test.values, degree=2)


Train the linear regression model

In [77]:
# Train the model
lr_poly_custom = LinearRegression()
lr = LinearRegression()
# fit the model
lr_poly_custom.fit(X_train_poly_custom, y_train)
lr.fit(X_train, y_train)
# predict values from the polynomial transformed features
predictions_poly_custom_train = lr_poly_custom.predict(X_train_poly_custom)
predictions_poly_custom = lr_poly_custom.predict(X_test_poly_custom)
# predict values from the original features
predictions_train = lr.predict(X_train)
predictions = lr.predict(X_test)

# mean squared error
print("Mean squared error (train poly custom): {:.2f}".format(mean_squared_error(y_train, predictions_poly_custom_train)))
print("Mean squared error (test poly custom): {:.2f}".format(mean_squared_error(y_test, predictions_poly_custom)))
print("Mean squared error (train): {:.2f}".format(mean_squared_error(y_train, predictions_train)))
print("Mean squared error (test): {:.2f}".format(mean_squared_error(y_test, predictions)))

# coefficient of determination (R^2)
print("R^2 (train poly custom): {:.2f}".format(r2_score(y_train, predictions_poly_custom_train)))
print("R^2 (test poly custom): {:.2f}".format(r2_score(y_test, predictions_poly_custom)))
print("R^2 (train): {:.2f}".format(r2_score(y_train, predictions_train)))
print("R^2 (test): {:.2f}".format(r2_score(y_test, predictions)))



Mean squared error (train poly custom): 53.87
Mean squared error (test poly custom): 50.78
Mean squared error (train): 107.67
Mean squared error (test): 105.88
R^2 (train poly custom): 0.81
R^2 (test poly custom): 0.82
R^2 (train): 0.61
R^2 (test): 0.63


RBFs on the California Housing Prices dataset

In [78]:
import numpy as np

# Implement the rbf_kernel() function
def rbf_kernel(X, centers, gamma):
    n_samples = X.shape[0]
    n_centers = centers.shape[0]
    K = np.zeros((n_samples, n_centers))

    for i in range(n_samples):
        for j in range(n_centers):
            diff = X[i] - centers[j]
            K[i, j] = np.exp(-gamma * np.dot(diff, diff))

    return K

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the California Housing Prices dataset
data = fetch_california_housing()
X = data['data']
y = data['target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=None)

# Standardize the data
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.fit_transform(X_test)

# Choose the number of centroids and the RBF kernel width
num_centroids = 100
gamma = 0.1

# Randomly select the centroids from the training set
np.random.seed(42)
idx = np.random.choice(X_train_std.shape[0], num_centroids, replace=False)
centroids = X_train_std[idx]

# Compute the RBF features for the training and testing sets
rbf_train = rbf_kernel(X_train_std, centroids, gamma)
rbf_test = rbf_kernel(X_test_std, centroids, gamma)

# Fit a linear regression model on the original and RBF-transformed data
linreg_orig = LinearRegression().fit(X_train_std, y_train)
linreg_rbf = LinearRegression().fit(rbf_train, y_train)

# Evaluate the models on the testing set
y_pred_orig = linreg_orig.predict(X_test_std)
mse_orig = mean_squared_error(y_test, y_pred_orig)
r2_orig = r2_score(y_test, y_pred_orig)

y_pred_rbf = linreg_rbf.predict(rbf_test)
mse_rbf = mean_squared_error(y_test, y_pred_rbf)
r2_rbf = r2_score(y_test, y_pred_rbf)

# Print the results
print("Linear regression on original data:")
print("MSE:", mse_orig)
print("R^2:", r2_orig)

print("\nLinear regression on RBF-transformed data:")
print("MSE:", mse_rbf)
print("R^2:", r2_rbf)

Linear regression on original data:
MSE: 0.5188091411147203
R^2: 0.6005669603670689

Linear regression on RBF-transformed data:
MSE: 0.5439916082749939
R^2: 0.5811788875554136


# **(Bonus)** Multilayer Perceptron Algorithm for Regression of Concrete Compressive Strength Dataset

Download the Concrete Compressive Strength Dataset from the UCI Machine Learning Repository.

In [ ]:
# Download the Concrete Compressive Strength Dataset from the UCI Machine Learning Repository.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import numpy as np

concrete = pd.read_excel('Concrete_Data.xls')

Preprocess the data

In [ ]:
# Selecting the Concrete compressive strength as targets, rest is features
X = concrete.drop('Concrete compressive strength(MPa, megapascals) ', axis=1)
y = concrete['Concrete compressive strength(MPa, megapascals) ']


# Preprocess the data
# --> maybe remove zeros and outliers


# Normalize the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
#X_test_std = scaler.fit_transform(X_test)

#Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.2, train_size=0.8, random_state=None, shuffle=True, stratify=None)

Define the multilayer perceptron algorithm

In [ ]:
import numpy as np

class MLP:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=0.01, epochs=1000):
        self.W1 = np.random.randn(input_dim, hidden_dim)
        self.b1 = np.zeros((1, hidden_dim))
        self.W2 = np.random.randn(hidden_dim, output_dim)
        self.b2 = np.zeros((1, output_dim))
        self.lr = lr
        self.epochs = epochs

    def forward(self, X):
        # ToDo: Implement the forward pass
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.y_hat = self.z2

    def backward(self, X, y):
        # ToDo: Implement the backward pass
        m = X.shape[0]
        delta2 = (1 / m) * (self.y_hat - y)
        dW2 = np.dot(self.a1.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        delta1 = np.dot(delta2, self.W2.T) * (1 - np.power(self.a1, 2))
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0)

        # Update parameters
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def fit(self, X, y):
        for epoch in range(self.epochs):
            self.forward(X)
            self.backward(X, y)

    def predict(self, X):
        self.forward(X)
        return self.y_hat.flatten()


Train the model

In [ ]:
# Create an instance of the MLP class
mlp = MLP(input_dim=X_train.shape[1], hidden_dim=10, output_dim=1, lr=0.01, epochs=1000)
# Train the model
mlp.fit(X_train, y_train)

ValueError: Data must be 1-dimensional, got ndarray of shape (824, 824) instead

Evaluate the model

In [ ]:
# Evaluate the model
y_pred = mlp.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

Mean Squared Error: 1625.7351921454845


Compare the results with the linear regression model

In [ ]:
# ToDo: fit a linear regression model on the training data

Mean Squared Error: 95.97548435337708
